<a href="https://colab.research.google.com/github/ktardthong/DSE_g19/blob/master/DSE_imageclassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Learning

##Pipeline
1. get data
2. extract feature
3. run model
4. test model
5. deploy model

In [ ]:
!pip install botnoi==0.2.1

     |████████████████████████████████| 16.0MB 292kB/s 


In [ ]:
from botnoi import scrape as sc
from botnoi import cv
import os

In [ ]:
def extractimagefeat(query):
  foldername = 'images/'+query
  isdir = os.path.isdir(foldername) 
  if not isdir:
    os.makedirs(foldername)
  #get images from google search
  imglist = sc.get_image_urls(query)
  i = 1
  for img in imglist[0:30]:
    #extract image features from each images and save to files
    try:
      print(i)
      savepath = foldername + '/' + str(i)+'.p'
      a = cv.image(img)
      a.getresnet50()
      a.save(savepath)
      i = i + 1
    except:
      pass
  return 'complete'


In [ ]:
import glob
import pandas as pd
import pickle
def createdataset(imgfolder):
  #imgfolder = glob.glob('images/*')
  dataset = []
  for cls in imgfolder:
    clsset = pd.DataFrame()
    pList = glob.glob(cls+'/*')
    featvec = []
    for p in pList:
      dat = pickle.load(open(p,'rb'))
      featvec.append(dat.restnet50)
    #featvec = np.vstack(featvec)
    clsset['feature'] = featvec
    cls = cls.split('/')[-1]
    clsset['label'] = cls
    dataset.append(clsset)
  return pd.concat(dataset,axis=0)

## Train model

In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV
def trainmodel(dataset,modfile=''):
  trainfeat, testfeat, trainlabel, testlabel = train_test_split(dataset['feature'], dataset['label'], test_size=0.33, random_state=42)
  clf = LinearSVC()
  clf = CalibratedClassifierCV(clf) 
  mod = clf.fit(np.vstack(trainfeat.values),trainlabel.values)
  res = mod.predict(np.vstack(testfeat.values))
  if modfile!='':
    pickle.dump(mod,open(modfile,'wb'))
  acc = sum(res == testlabel.values)/len(res)
  return mod,acc

In [ ]:
#pipeline to create model
clsList = ['หมา','แมว']
for c in clsList:
  extractimagefeat(c)
imgfolder = ['images/'+c for c in clsList]
dataset = createdataset(imgfolder)
#dataset
modFile = 'mymodel.p'
mod,acc = trainmodel(dataset,modFile)

complete: 0.00%
completed
1
94773248/94765736 [==============================] - 1s 0us/step
2
3
4
5
6
7
8
9
10
11
12
13
14
15
15
15
16
17
18
19
20
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
37
38
39
40
41
42
42
43
44
44
45
46
47
48
49
50
51
52
53
54
55
56
complete: 0.00%
completed
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
48
49
50
51
52
53
54
55
56
57
58


AttributeError: ignored

In [ ]:
# output function
mymod = pickle.load(open(modFile,'rb'))
def predictimg(imgurl):
  a = cv.image(imgurl)
  feat = a.getresnet50()
  probList = mod.predict_proba([feat])[0]
  maxprobind = np.argmax(probList)
  prob = probList[maxprobind]
  outclass = mymod.classes_[maxprobind]
  result = {}
  result['class'] = outclass
  result['probability'] = prob
  return result


In [ ]:
a=predictimg('https://do.lnwfile.com/oddu08.jpg')
a

{'class': 'หมา', 'probability': 0.8503068726898757}

In [ ]:
predictimg('https://cf.shopee.co.th/file/f338f8d43a8c609c9518d8a4a9915947')

(CalibratedClassifierCV(base_estimator=LinearSVC(C=1.0, class_weight=None,
                                                 dual=True, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 loss='squared_hinge',
                                                 max_iter=1000,
                                                 multi_class='ovr', penalty='l2',
                                                 random_state=None, tol=0.0001,
                                                 verbose=0),
                        cv=None, method='sigmoid'), 0)